In [39]:
# XOR 예시
import tensorflow as tf
import pandas as pd
import numpy as np

#inputX = [[0, 0], [0, 1], [1, 0], [1, 1]]   # input 데이터
#outputY = [[1], [0], [1], [0]]              # desired output 데이터

In [40]:
sm = pd.read_csv('./stockData/005930.csv', index_col=0, parse_dates=True)
sm = sm.sort_index() # 날짜 순으로 정렬한다
sm = sm.dropna()
sm = sm / 10000

In [41]:
sm.head()

,open,high,low,close,volume
date,,,,,
2011-05-12,1.772,1.802,1.766,1.770,55.2942
2011-05-13,1.792,1.848,1.780,1.832,66.9154
2011-05-16,1.834,1.838,1.796,1.802,32.2905
2011-05-17,1.794,1.796,1.778,1.780,23.1293
2011-05-18,1.786,1.798,1.772,1.786,33.1824


In [42]:
sm.tail()

,open,high,low,close,volume
date,,,,,
2019-06-19,4.545,4.545,4.500,4.535,1041.7690
2019-06-20,4.485,4.550,4.485,4.550,714.7785
2019-06-21,4.575,4.580,4.520,4.570,945.4913
2019-06-24,4.520,4.580,4.520,4.550,608.5066
2019-06-25,4.520,4.580,4.520,4.545,412.0726


In [43]:
sm['next close'] = sm['close'].shift(-1)

In [44]:
sm.head()

,open,high,low,close,volume,next close
date,,,,,,
2011-05-12,1.772,1.802,1.766,1.770,55.2942,1.832
2011-05-13,1.792,1.848,1.780,1.832,66.9154,1.802
2011-05-16,1.834,1.838,1.796,1.802,32.2905,1.780
2011-05-17,1.794,1.796,1.778,1.780,23.1293,1.786
2011-05-18,1.786,1.798,1.772,1.786,33.1824,1.760


In [45]:
sm.tail()

,open,high,low,close,volume,next close
date,,,,,,
2019-06-19,4.545,4.545,4.500,4.535,1041.7690,4.550
2019-06-20,4.485,4.550,4.485,4.550,714.7785,4.570
2019-06-21,4.575,4.580,4.520,4.570,945.4913,4.550
2019-06-24,4.520,4.580,4.520,4.550,608.5066,4.545
2019-06-25,4.520,4.580,4.520,4.545,412.0726,NaN


In [46]:
testX = np.array(sm.iloc[-1:, 0:4])
testX

array([[4.52 , 4.58 , 4.52 , 4.545]])

In [47]:
sm = sm.dropna()

In [48]:
inputX = np.array(sm.iloc[:, 0:4])
inputX.shape

(1999, 4)

In [49]:
outputY = np.array(sm.loc[:, 'next close']).reshape(-1,1)

In [50]:
outputY

array([[1.832],
       [1.802],
       [1.78 ],
       ...,
       [4.57 ],
       [4.55 ],
       [4.545]])

In [51]:
nInput = 4      # input layer의 neuron 개수
nHidden1 = 60   # hidden layer-1의 neuron 개수
nHidden2 = 60   # hidden layer-2의 neuron 개수 
nOutput = 1     # output layer의 neuron 개수

In [63]:
# 그래프를 생성한다
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[None, nInput], name='x')
y = tf.placeholder(tf.float32, shape=[None, nOutput], name='y')

# hidden layer의 Weight (Wh)와 Bias (Bh)
Wh1 = tf.Variable(tf.truncated_normal([nInput, nHidden1]), dtype=tf.float32, name='Wh1')
Bh1 = tf.Variable(tf.zeros(nHidden1), dtype=tf.float32, name='Bh1')

# hidden layer-2의 Weight (Wh)와 Bias (Bh)
Wh2 = tf.Variable(tf.truncated_normal([nHidden1, nHidden2]), dtype=tf.float32, name='Wh2')
Bh2 = tf.Variable(tf.zeros(nHidden2), dtype=tf.float32, name='Bh2')

# output layer의 Weight (Wo)와 Bias (Bo)
Wo = tf.Variable(tf.truncated_normal([nHidden2, nOutput]), dtype=tf.float32, name='Wo')
Bo = tf.Variable(tf.zeros(nOutput), dtype=tf.float32, name='Bo')

# Hidden layer의 출력값. activation function은 sigmoid
H1 = tf.nn.relu(tf.matmul(x, Wh1) + Bh1, name='H1')
H2 = tf.nn.relu(tf.matmul(H1, Wh2) + Bh2, name='H2')

# Output layer의 출력값. activation function은 sigmoid
predY = tf.add(tf.matmul(H2, Wo), Bo, name='predY')

# Cost function 정의. MSE
cost = tf.reduce_mean(tf.square(tf.subtract(predY, y)))

# 학습
alpha = tf.placeholder(tf.float32, shape=(), name='alpha')
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cost)

In [62]:
H2

<tf.Tensor 'H2:0' shape=(4, 60) dtype=float32>

In [64]:
# 그래프를 실행한다
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [76]:
# 인공신경망에 inputX, outputY를 1000번 집어 넣어서 학습 시킨다.
for i in range(10000):
    _, xcost = sess.run([train, cost], feed_dict={x: inputX, y: outputY, alpha:0.0001})
    
    if i % 1000 == 0:
        print('cost = %.6f' % xcost)

cost = 0.002984
cost = 0.002964
cost = 0.002965
cost = 0.002965
cost = 0.002964
cost = 0.002964
cost = 0.002966
cost = 0.002967
cost = 0.002974
cost = 0.002963


In [74]:
testX

array([[4.52 , 4.58 , 4.52 , 4.545]])

In [77]:
# 학습이 완료되면, Wh, Bh, Wo, Bo 이 모두 업데이트 되었으면, inputX를 넣어서 출력값을 확인한다.
# 출력값은 inputY의 추정값이다.
yHat = sess.run(predY, feed_dict={x: testX})

print("\ny-추정치 = \n", yHat * 10000)
#sess.close()


y-추정치 = 
 [[45567.43]]
